<a href="https://colab.research.google.com/github/rajeshaouti/DeepLearning/blob/master/Quiz9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self, dropout_value=0.05, input_ch=3):
        super(Net, self).__init__()
        self.dropout_value = dropout_value
        self.input_ch = input_ch

        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=input_ch, out_channels=10, kernel_size=(
                3, 3), padding=1, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )  # input =3, output=10 channels

        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=13, out_channels=16, kernel_size=(
                3, 3), padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )

        self.pool1 = nn.MaxPool2d(2, 2)  # output_size = 11

        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=29, out_channels=32, kernel_size=(
                3, 3), padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=61, out_channels=64, kernel_size=(
                3, 3), padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )

        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=125, out_channels=128, kernel_size=(
                3, 3), padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )

        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=224, out_channels=256, kernel_size=(
                3, 3), padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )

        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=480, out_channels=512, kernel_size=(
                3, 3), padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )

        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=992, out_channels=1024, kernel_size=(
                3, 3), padding=1, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )

        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=8))

        self.fc = nn.Sequential(nn.Linear(10, 10))

        self.convblock10 = nn.Sequential(
            nn.Conv2d(in_channels=1024, out_channels=10, kernel_size=(
                1, 1), padding=0, bias=False),
            # nn.BatchNorm2d(10),
            # nn.ReLU()
        )  # output_size = 5

    

    def forward(self, x):
        x2 = self.convblock1(x)
        x3 = self.convblock2(
            torch.cat((x, x2), dim=1))  # ip = 13, op=16
        # ch=32 size =16
        # ip 32, op32
        x4 = self.pool1(torch.cat((x, x2, x3), dim=1))
        x5 = self.convblock3(x4)  # inp 32, op=32
        x6 = self.convblock4(
            torch.cat((x4, x5), dim=1))  # ip 64, op 64
        x7 = self.convblock5(
            torch.cat((x4, x5, x6), dim=1))  # ip 128, op 128

        x8 = self.pool1(
            torch.cat((x5, x6, x7), dim=1))  # 224
        x9 = self.convblock6(x8)  # inp 224, op 256
        x10 = self.convblock7(
            torch.cat((x8, x9), dim=1))  # 480 op 512
        x11 = self.convblock8(
            torch.cat((x8, x9, x10), dim=1))  # 1024
        x12 = self.gap(x11)
        x13 = self.convblock10(x12)
        x14 = x13.view(-1, 10)  # 10
        x15 = self.fc(x14)
        return F.log_softmax(x15, dim=1)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 32, 32]             270
       BatchNorm2d-2           [-1, 10, 32, 32]              20
              ReLU-3           [-1, 10, 32, 32]               0
           Dropout-4           [-1, 10, 32, 32]               0
            Conv2d-5           [-1, 16, 32, 32]           1,872
       BatchNorm2d-6           [-1, 16, 32, 32]              32
              ReLU-7           [-1, 16, 32, 32]               0
           Dropout-8           [-1, 16, 32, 32]               0
         MaxPool2d-9           [-1, 29, 16, 16]               0
           Conv2d-10           [-1, 32, 16, 16]           8,352
      BatchNorm2d-11           [-1, 32, 16, 16]              64
             ReLU-12           [-1, 32, 16, 16]               0
          Dropout-13           [-1, 32, 16, 16]               0
           Conv2d-14           [-1, 64,

In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize([0.49139968, 0.48215841, 0.44653091], 
                                              [0.24703223, 0.24348513, 0.26158784])
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize([0.49139968, 0.48215841, 0.44653091], 
                                              [0.24703223, 0.24348513, 0.26158784])
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/cifar-10-python.tar.gz to ../data


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 1.5113, Accuracy: 4584/10000 (45.84%)



  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.9759, Accuracy: 6566/10000 (65.66%)



  0%|          | 0/391 [00:00<?, ?it/s]

...


Test set: Average loss: 1.0305, Accuracy: 7607/10000 (76.07%)



  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.9247, Accuracy: 7841/10000 (78.41%)



loss=0.1392136812210083 batch_id=390: 100%|██████████| 391/391 [00:30<00:00, 13.02it/s]



Test set: Average loss: 0.8445, Accuracy: 8049/10000 (80.49%)

